# Creación de Pandas DataFrames a partir de peticiones API
En este ejemplo, utilizaremos la API del U.S. Geological Survey para obtener un objeto JSON de datos de terremotos y convertirlo en un `pandas.DataFrame`.

API DEL USGS: https://earthquake.usgs.gov/fdsnws/event/1/

### Obtener datos de la API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# asegurémonos de que la solicitud fue correcta
response.status_code

200

Una respuesta de 200 significa OK, así que podemos extraer los datos del resultado. Como le pedimos a la API una carga JSON, podemos extraerla de la respuesta con el método `json()`.

### Aislar los datos de la respuesta JSON
Necesitamos comprobar las estructuras de los datos de la respuesta para saber dónde están nuestros datos.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

La API del USGS proporciona información sobre nuestra solicitud en la clave `metadata`. Tenga en cuenta que el resultado será diferente, independientemente del intervalo de fechas que elija, ya que la API incluye una marca de tiempo que indica cuándo se extrajeron los datos:

In [3]:
earthquake_json['metadata']

{'generated': 1699703328000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-10-11&endtime=2023-11-10',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.0',
 'count': 10168}

Cada elemento de la matriz JSON `features` es una fila de datos para nuestro marco de datos.

In [4]:
type(earthquake_json['features'])

list

Sus datos serán diferentes dependiendo de la fecha en que ejecute esto.

In [9]:
earthquake_json['features']

[{'type': 'Feature',
  'properties': {'mag': 1.6,
   'place': 'northern Alaska',
   'time': 1699574320362,
   'updated': 1699574475288,
   'tz': None,
   'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak023ee194el',
   'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak023ee194el&format=geojson',
   'felt': None,
   'cdi': None,
   'mmi': None,
   'alert': None,
   'status': 'automatic',
   'tsunami': 0,
   'sig': 39,
   'net': 'ak',
   'code': '023ee194el',
   'ids': ',ak023ee194el,',
   'sources': ',ak,',
   'types': ',origin,phase-data,',
   'nst': None,
   'dmin': None,
   'rms': 1.14,
   'gap': None,
   'magType': 'ml',
   'type': 'earthquake',
   'title': 'M 1.6 - northern Alaska'},
  'geometry': {'type': 'Point', 'coordinates': [-146.6613, 65.226, 21.1]},
  'id': 'ak023ee194el'},
 {'type': 'Feature',
  'properties': {'mag': 0.68,
   'place': '2 km NNW of The Geysers, CA',
   'time': 1699574010710,
   'updated': 1699574531658,
   'tz': None,
   'url'

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.6,
  'place': 'northern Alaska',
  'time': 1699574320362,
  'updated': 1699574475288,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak023ee194el',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak023ee194el&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 39,
  'net': 'ak',
  'code': '023ee194el',
  'ids': ',ak023ee194el,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 1.14,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.6 - northern Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-146.6613, 65.226, 21.1]},
 'id': 'ak023ee194el'}

### Convertir a DataFrame
Necesitamos tomar la sección `properties` de cada entrada del array JSON `features` para crear nuestro dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.60,northern Alaska,1699574320362,1699574475288,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak023ee194el,",",ak,",",origin,phase-data,",NaN,NaN,1.14,NaN,ml,earthquake,M 1.6 - northern Alaska
1,0.68,"2 km NNW of The Geysers, CA",1699574010710,1699574531658,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73960686,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",10.0,0.009218,0.02,76.0,md,earthquake,"M 0.7 - 2 km NNW of The Geysers, CA"
2,1.50,"38 km WSW of Mentone, Texas",1699573142754,1699575826892,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",tx2023vzvu,",",tx,",",origin,phase-data,",15.0,0.000000,0.20,82.0,ml(texnet),earthquake,"M 1.5 - 38 km WSW of Mentone, Texas"
3,2.23,"8 km ENE of Pāhala, Hawaii",1699572964970,1699573166570,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73642497,",",hv,",",origin,phase-data,",36.0,NaN,0.12,150.0,md,earthquake,"M 2.2 - 8 km ENE of Pāhala, Hawaii"
4,0.27,"7 km W of Cobb, CA",1699572653460,1699573210375,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73960676,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",10.0,0.010430,0.01,74.0,md,earthquake,"M 0.3 - 7 km W of Cobb, CA"


In [10]:
type(earthquake_properties_data)

list

In [11]:
earthquake_properties_data

[{'mag': 1.6,
  'place': 'northern Alaska',
  'time': 1699574320362,
  'updated': 1699574475288,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak023ee194el',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak023ee194el&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 39,
  'net': 'ak',
  'code': '023ee194el',
  'ids': ',ak023ee194el,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 1.14,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.6 - northern Alaska'},
 {'mag': 0.68,
  'place': '2 km NNW of The Geysers, CA',
  'time': 1699574010710,
  'updated': 1699574531658,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc73960686',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc73960686&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi'

### (Opcional) Guardar los datos en un CSV

In [12]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creando_dataframes.ipynb">
        <button style="float: left;">&#8592; Notebook Anterior</button>
    </a>
    <a href="./4-inspeccionando_dataframes.ipynb">
        <button style="float: right;">Siguiente Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>